# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [121]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [122]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [123]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [201]:
# Define Panel Visualization Functions
def housing_units():
    """Housing Units Per Year."""
    
    # YOUR CODE HERE!
    # OG CODE mean_data = sfo_data.groupby('year').mean()
    
    mean_data = sfo_data['housing_units'].groupby(sfo_data.index).mean()

    fig_housing_units = plt.figure()
    plot_housing_units = mean_data.plot.bar(y='housing_units')
    plot_housing_units.set_xlabel("Year")
    plot_housing_units.set_ylabel("Housing Units")
    plot_housing_units.set_title("Housing Units in San Francisco from 2010 to 2016")
    plot_housing_units.set_ylim(370000,385000)
    
    plt.close(fig_housing_units)

    return pn.pane.Matplotlib(fig_housing_units, tight=True)

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""

    # YOUR CODE HERE!
    mean_data = sfo_data[['sale_price_sqr_foot','gross_rent']].groupby(sfo_data.index).mean()
    
    fig_gross_rent = plt.figure()
    plot_gross_rent = mean_data['gross_rent'].plot.line(y='gross_rent')
    plot_gross_rent.set_xlabel("Year")
    plot_gross_rent.set_ylabel("Gross Rent")
    plot_gross_rent.set_title("Average Gross Rent in San Francisco from 2010 to 2016")
    
    plt.close(fig_gross_rent)

    return pn.pane.Matplotlib(fig_gross_rent, tight=True)

def average_sales_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    mean_data = sfo_data[['sale_price_sqr_foot','gross_rent']].groupby(sfo_data.index).mean()

    fig_sales_price = plt.figure()
    plot_sales_price = mean_data['sale_price_sqr_foot'].plot.line(y='sale_price_sqr_foot')
    plot_sales_price.set_xlabel("Year")
    plot_sales_price.set_ylabel("Average Sale Price / Sq. Ft.")
    plot_sales_price.set_title("Average Sale Price / Sq. Ft. From 2010 to 2016")
    plt.close(fig_sales_price)
     
    return pn.pane.Matplotlib(fig_sales_price)

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    grouped_df=sfo_data.groupby(['year','neighborhood'])
    mean_grouped_df=grouped_df.mean().reset_index()

    price_by_neighborhood = mean_grouped_df.hvplot.line(x='year', y='sale_price_sqr_foot', groupby='neighborhood', xlabel='Year', ylabel='Average Sale Price / Sq. Ft.')

    return price_by_neighborhood
    

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    # YOUR CODE HERE!
    neighborhood_df=sfo_data.groupby('neighborhood').mean()
    ten_most_exp = neighborhood_df.nlargest(10,'sale_price_sqr_foot').reset_index()
    
    top_ten_exp = ten_most_exp.hvplot.bar(x='neighborhood', y='sale_price_sqr_foot',rot=90, xlabel='Neighborhood', ylabel='Average Sale Price / Sq. Ft.')

    return top_ten_exp

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    # YOUR CODE HERE!
    
    neighborhood_df=sfo_data.groupby('neighborhood').mean()
    ten_most_exp = neighborhood_df.nlargest(10,'sale_price_sqr_foot').reset_index()
    parallel_coordinates_plot = px.parallel_coordinates(ten_most_exp, color='sale_price_sqr_foot')

    return parallel_coordinates_plot

def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!

    neighborhood_df=sfo_data.groupby('neighborhood').mean()
    ten_most_exp = neighborhood_df.nlargest(10,'sale_price_sqr_foot').reset_index()
    parallel_categories_plot = px.parallel_categories(
    ten_most_exp, dimensions=['neighborhood','sale_price_sqr_foot','housing_units','gross_rent'],  color='sale_price_sqr_foot', color_continuous_scale=px.colors.sequential.Inferno)
    
    return parallel_categories_plot

def neighborhood_map():
    """Neighborhood Map"""
    
    # YOUR CODE HERE!

    all_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    all_neighborhoods.reset_index(inplace=True)
    all_neighborhoods.rename(columns={"neighborhood": "Neighborhood"}, inplace=True)

    all_neighborhoods = pd.merge(df_neighborhood_locations, all_neighborhoods)

map = px.scatter_mapbox(
    all_neighborhoods,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="gross_rent",
    zoom=10,
    hover_name="Neighborhood",
    color_continuous_scale=px.colors.cyclical.IceFire,
    title="San Francisco Neighborhood Average Sale Price / Sq. Ft. and Gross Rent")

    return map

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [204]:
# YOUR CODE HERE!
title = pn.pane.Markdown("")

welcome = pn.pane.Markdown("This dashboard shows a visual representation of San Francisco housing units from 2010-2016, and the sale price per square foot and gross rent of different neighborhoods in the San Francisco area.")

tabs = pn.Tabs(
    ("Welcome", pn.Column(welcome, neighborhood_map())),
    ("Yearly Market Analysis", pn.Column(housing_units(), average_gross_rent(), average_sales_price())),
    ("Neighborhood Analysis", pn.Column(average_price_by_neighborhood(), top_most_expensive_neighborhoods())),
    (
        "Parallel Plots Analysis",
        pn.Column(parallel_coordinates(), parallel_categories(), width=960),
    ),
)


dashboard = pn.Column(pn.Row(title), tabs)

## Serve the Panel Dashboard

In [203]:
dashboard.show()

Launching server at http://localhost:59778
